

# P3
Note, for the midterm we would not have this here, we would have them implement it from scratch, but I'm leaving it here for you to see

In [1]:
import os
os.getcwd()

'/Users/sven/repos/stanford/teaching/cme-241/RL-book/rl/midterm_2022'

In [2]:
from dataclasses import dataclass
from rl import dynamic_programming

In [3]:
import operator
from typing import Mapping, Iterator, TypeVar, Tuple, Dict, Iterable

import numpy as np

from rl.distribution import Categorical, Choose
from rl.iterate import converged, iterate
from rl.markov_process import NonTerminal, State
from rl.markov_decision_process import (FiniteMarkovDecisionProcess,
                                        FiniteMarkovRewardProcess)
from rl.policy import FinitePolicy, FiniteDeterministicPolicy

In [4]:
@dataclass(frozen=True)
class GameState:
    on_board: Tuple[int]
    in_hand: Tuple[int]
        
    def score(self, criterion):
        if criterion(self.in_hand) and sum(self.on_board) == 0:
            return sum([v * x for v, x in zip(range(1, len(self.in_hand)+1), self.in_hand)])
        else:
            return 0

@dataclass(frozen=True)
class GameStateAction:
    moves_to_hand: Tuple[int]

In [5]:
import math

In [6]:
from itertools import product
class DiceGameMDP(FiniteMarkovDecisionProcess[GameState, int]):

    def __init__(self, num_dice, dice_faces, min_ones):
        self.dice_faces = dice_faces
        self.num_dice = num_dice
        self.criterion = lambda x: x[0] >= min_ones
        super().__init__(self.get_action_transition_reward_map())

    def get_action_transition_reward_map(self):
        d: Dict[GameState, Dict[GameStateAction, Categorical[Tuple[GameState,
                                                            float]]]] = {}

        
        for state in filter(lambda x: sum(x) == self.num_dice and sum(x[:self.dice_faces]) > 0, 
                            product(*[range(self.num_dice + 1) for _ in range(self.dice_faces*2)])):
            
            board_state = tuple(x for x in state[:self.dice_faces])
            hand_state = tuple(x for x in state[self.dice_faces:])
            die_on_board = sum(board_state)
            state: InventoryState = GameState(board_state, hand_state)
            
            assert sum(board_state) > 0
                
            d1: Dict[GameStateAction, Categorical[Tuple[GameState, float]]] = {}
            for move in filter(lambda x: sum(x) > 0,
                              product(*[range(ob + 1) for ob in board_state])):
                action = GameStateAction(move)
                new_hand_state = tuple(x + y for x, y in zip(hand_state, move))
                die_remaining = die_on_board - sum(move)
                d2: dict[Tuple[GameState, float], float] = {}
                for new_board_state in filter(lambda x: sum(x) == die_remaining, 
                                              product(*[range(die_remaining + 1) 
                                                        for _ in range(self.dice_faces)])):
                    next_state = GameState(new_board_state, new_hand_state)
                    prod = 1
                    curr_sum = sum(next_state.on_board)
                    for s in next_state.on_board:
                        prod *= math.comb(curr_sum, s)
                        curr_sum -= s
                    d2[(next_state, next_state.score(self.criterion))] = prod / (max(
                        sum(next_state.on_board)**self.dice_faces, 1))
                d1[action] = Categorical(d2)
            d[state] = d1
        return d


In [7]:
import time
start_time = time.time()

In [8]:
mdp = DiceGameMDP(6, 4, 1)

In [9]:
starting_states = [x for x in mdp.non_terminal_states if sum(x.state.in_hand) == 0]

In [10]:
vf, pol = dynamic_programming.value_iteration_result(mdp, 1)

In [11]:
average_score = np.mean([vf[s] for s in starting_states])
average_score

18.3191397712189

In [12]:
s = GameState(on_board=(1,2,2,1), in_hand=(0, 0, 0, 0))
pol.action_for[s], vf[NonTerminal(state=s)]

(GameStateAction(moves_to_hand=(0, 0, 0, 1)), 18.376703715883192)

In [13]:
print(time.time() - start_time)

5.114749908447266


# Note
Disregard the code below this, this is just if you want to see how you would play the game 

In [15]:
import random

In [16]:
def play_policy(mdp, policy):
    start = random.choice([x for x in mdp.non_terminal_states if sum(x.state.in_hand) == 0]).state
    curr_state = start
    while sum(curr_state.on_board) > 0:
        print(curr_state)
        move = policy.action_for[curr_state].moves_to_hand
        print(move)
        action = GameStateAction(move)
        curr_state = mdp.mapping[NonTerminal(curr_state)][action].sample()[0].state
        print()
    return curr_state.score(mdp.criterion)

In [17]:
def play(mdp):
    start = random.choice([x for x in mdp.non_terminal_states if sum(x.state.in_hand) == 0]).state
    curr_state = start
    while sum(curr_state.on_board) > 0:
        print(curr_state)
        move = tuple(int(x) for x in input().split(','))
        print(move)
        action = GameStateAction(move)
        curr_state = mdp.mapping[NonTerminal(curr_state)][action].sample()[0].state
        print()
    return curr_state.score(mdp.criterion)

In [18]:
play(mdp)

GameState(on_board=(1, 0, 3, 2), in_hand=(0, 0, 0, 0))


KeyboardInterrupt: Interrupted by user

In [ ]:
play_policy(mdp, pol)

In [ ]:
import random.score

In [ ]:
start = random.choice([x for x in mdp.non_terminal_states if sum(x.state.in_hand) == 0]).state